In [2]:
import os
import pandas as pd
from arosics import COREG, COREG_LOCAL
from zipfile import ZipFile
from IPython.display import Image
from rasterio.features import bounds, dataset_features
import sys
sys.path.insert(1, '../')
from pyproj import Proj
from utils import *

%matplotlib inline

* Time series handling is limited. mainly pairwise. Need coding to loop trhough series.
* Takes 30 secs to match two full res S2 scenes in global mode
* Takse about 3 mins to match two full res S2 scenes with 120 grid points and running on 96 cores! (A bit concerning)
* Also running in full res for local matching is very resource intensive!
* Documentation not very straightforward!
* Seems to be partially handaling the cloud cover.
* AU scenes are already co-registered mostly.
* Local matching will fail for small overlaps.
* window size of (500, 500) recommended. 
* overall both S1 and S2 register good over AU
* local matching is recommended

In [3]:
DATA_DIR = "../data/files_list"

S1_PRODUCTS = ["SLC", "GRD"]
S2_PRODUCTS = ["L1C", "L2A"]

s1_au_df = pd.read_csv(os.path.join(DATA_DIR, "s1_au.csv"), names=["ID","Path"])
s1_an_df = pd.read_csv(os.path.join(DATA_DIR, "s1_an.csv"), names=["ID","Path"])

s2_au_df = pd.read_csv(os.path.join(DATA_DIR, "s2_au.csv"), names=["ID","Path"])
s2_an_df = pd.read_csv(os.path.join(DATA_DIR, "s2_an.csv"), names=["ID","Path"])

s1_au_slc_dict = get_scenes_dict(s1_au_df, "SLC")
s1_au_grd_dict = get_scenes_dict(s1_au_df, "GRD")

s1_an_slc_dict = get_scenes_dict(s1_an_df, "SLC")
s1_an_grd_dict = get_scenes_dict(s1_an_df, "GRD")

s2_au_l1c_dict = get_scenes_dict(s2_au_df, "L1C", False)
s2_au_l2a_dict = get_scenes_dict(s2_au_df, "L2A", False)

s2_an_l1c_dict = get_scenes_dict(s2_an_df, "L1C", False)
s2_an_l2a_dict = get_scenes_dict(s2_an_df, "L2A", False)


In [4]:
shutil.rmtree("../data/inputs/", ignore_errors=True)
# shutil.rmtree("../data/outputs/", ignore_errors=True)
os.makedirs("../data/inputs/")
os.makedirs("../data/outputs/", exist_ok = True)

##### Test two S2 L2A images for AN

In [106]:

id = list(s2_an_l2a_dict.keys())[1]
s2_an_l2a_secne_files = s2_an_l2a_dict[id]

# get the Jan 2023 and Dec 2024 files
s2_an_l2a_secne_pair = [s2_an_l2a_secne_files[10], s2_an_l2a_secne_files[-1]]

shutil.rmtree(f"../data/inputs/{id}/", ignore_errors=True)
for i, zip_file_path in enumerate(s2_an_l2a_secne_pair):
    with ZipFile(zip_file_path) as f:
        f.extractall(f"../data/inputs/{id}/sub{i}")
ref_image_dir = os.path.join("../data/inputs/", id, f"sub{0}")
ref_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
ref_tci_files = list(filter(lambda f: "L2A" in f, ref_tci_files))
if len(ref_tci_files) > 1:
    ref_image = [f for f in ref_tci_files if f.endswith("_10m.jp2")][0]
else:
    ref_image = ref_tci_files[0]

tgt_image_dir = os.path.join("../data/inputs/", id, f"sub{1}")
tgt_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{tgt_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
tgt_tci_files = list(filter(lambda f: "L2A" in f, tgt_tci_files))
if len(tgt_tci_files) > 1:
    tgt_image = [f for f in tgt_tci_files if f.endswith("_10m.jp2")][0]
else:
    tgt_image = tgt_tci_files[0]

global_output = os.path.join("../data/outputs/", "L2A_pair", f"{id}_global.tiff")
local_output = os.path.join("../data/outputs/", "L2A_pair", f"{id}_local.tiff")

In [ ]:
%%time
coreg_global = COREG(
    im_ref=tgt_image,
    im_tgt=ref_image,
    path_out=global_output,
    fmt_out="GTIFF",
    # v=True,
    nodata=(0.0,0.0),
    r_b4match=2,
    s_b4match=2,
    align_grids=True,
    max_iter = 10,
    max_shift = 10,
)
res = coreg_global.correct_shifts()

In [ ]:
%%time
coreg_local = COREG_LOCAL(
    im_ref=tgt_image,
    im_tgt=ref_image,
    grid_res=1000,
    # max_points=200,
    path_out=local_output,
    fmt_out="GTIFF",
    # v=True,
    nodata=(0.0,0.0),
    # r_b4match=2,
    # s_b4match=2,
    align_grids=True,
    max_iter = 10,
    max_shift = 10,
    CPUs = 8,
)
res = coreg_local.correct_shifts()

In [ ]:
datasets_paths = [tgt_image, ref_image,  local_output]
dataset_titles = ["Reference", "Target", "Local matching"]
make_difference_gif(datasets_paths, "s2_l2a_pair.gif", dataset_titles, 0.1)
Image(url='s2_l2a_pair.gif', width = 400, height=400) 

##### Test two S2 L1C secenes

In [19]:
id = list(s2_an_l1c_dict.keys())[0]
s2_an_l1c_secne_files = s2_an_l1c_dict[id]

# get the Jan 2023 and Dec 2024 files
s2_an_l1c_secne_pair = [s2_an_l1c_secne_files[0], s2_an_l1c_secne_files[-1]]

shutil.rmtree(f"../data/inputs/{id}/", ignore_errors=True)
for i, zip_file_path in enumerate(s2_an_l1c_secne_pair):
    with ZipFile(zip_file_path) as f:
        f.extractall(f"../data/inputs/{id}/sub{i}")
ref_image_dir = os.path.join("../data/inputs/", id, f"sub{0}")
ref_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
ref_tci_files = list(filter(lambda f: "L1C" in f, ref_tci_files))
if len(ref_tci_files) > 1:
    ref_image = [f for f in ref_tci_files if f.endswith("_10m.jp2")][0]
else:
    ref_image = ref_tci_files[0]

tgt_image_dir = os.path.join("../data/inputs/", id, f"sub{1}")
tgt_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{tgt_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
tgt_tci_files = list(filter(lambda f: "L1C" in f, tgt_tci_files))
if len(tgt_tci_files) > 1:
    tgt_image = [f for f in tgt_tci_files if f.endswith("_10m.jp2")][0]
else:
    tgt_image = tgt_tci_files[0]

local_output = os.path.join("../data/outputs/", "L1C_pair", f"{id}_local.tiff")
global_output = os.path.join("../data/outputs/", "L1C_pair", f"{id}_global.tiff")

In [ ]:
%%time
coreg_global = COREG(
    im_ref=ref_image,
    im_tgt=tgt_image,
    path_out=global_output,
    fmt_out="GTIFF",
    # v=True,
    nodata=(0.0,0.0),
    r_b4match=2,
    s_b4match=2,
    align_grids=True,
    max_iter = 10,
    max_shift = 10,
    ws = (500, 500),
)
res = coreg_global.correct_shifts()

In [ ]:
%%time
coreg_local = COREG_LOCAL(
    im_ref=ref_image,
    im_tgt=tgt_image,
    grid_res=1000,
    # max_points=200,
    path_out=local_output,
    fmt_out="GTIFF",
    # v=True,
    nodata=(0.0,0.0),
    # r_b4match=2,
    # s_b4match=2,
    align_grids=True,
    max_iter = 10,
    max_shift = 10,
    CPUs = 8,
)
res = coreg_local.correct_shifts()

In [ ]:
datasets_paths = [ref_image, tgt_image, local_output]
datasets_titles = ["Reference", "Target", "Local matching"]
make_difference_gif(datasets_paths, "s2_l1c_an_pair.gif", datasets_titles, 0.1)
Image(url='s2_l1c_an_pair.gif', width = 400, height=400) 

##### S2 L1C and S1 RTC AN

In [5]:
# find scenes
s2_id = list(s2_an_l1c_dict.keys())[0]
s2_an_l1c_secne_files = s2_an_l1c_dict[s2_id]

# shutil.rmtree(f"../data/inputs/{s2_id}/", ignore_errors=True)

# get the Jan 2023 for S2 L1C
s2_an_l1c_scene = s2_an_l1c_secne_files[0]
with ZipFile(s2_an_l1c_scene) as f:
    f.extractall(f"../data/inputs/{s2_id}/sub0")

ref_image_dir = os.path.join("../data/inputs/", s2_id, f"sub{0}")
ref_tci_files = list(filter(lambda f: "TCI" in f, glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True)))
ref_tci_files = list(filter(lambda f: "L1C" in f, ref_tci_files))
if len(ref_tci_files) > 1:
    ref_image = [f for f in ref_tci_files if f.endswith("_10m.jp2")][0]
else:
    ref_image = ref_tci_files[0]

ref = rasterio.open(ref_image)
ref_bounds = ref.bounds
ref_crs = ref.crs

ref_proj = Proj(**ref_crs.data)

west, south = ref_proj(ref_bounds.left, ref_bounds.bottom, inverse = True)
east, north = ref_proj(ref_bounds.right, ref_bounds.top, inverse = True)

bbox = f"{west},{south},{east},{north}"

os.makedirs("../data/asf/sub1", exist_ok=True)
with open(f"../data/asf/sub1/bbox.txt", "w") as f:
    f.write(f"{bbox}\n")
    f.write(f"{s2_id}\n")



In [145]:
assert os.path.isfile(f"../data/inputs/{s2_id}/sub1/rtc.txt"), "RTC file for the scene not readt or none-existent. Run `rtc_job_single_scene` notebook."
with open(f"../data/inputs/{s2_id}/sub1/rtc.txt", "r") as f:
    rtc_file = f.readline()

with ZipFile(rtc_file) as f:
    f.extractall(f"../data/inputs/{s2_id}/sub1")
    
tgt_image_dir = os.path.join("../data/inputs/", s2_id, "sub1", f"{os.path.splitext(os.path.basename(rtc_file))[0]}")
tgt_image = list(filter(lambda f: ("rgb" in f) and f.endswith(".tif"), glob.glob(f"{tgt_image_dir}/*")))[0]
# repreojected_tgt_path = os.path.join(os.path.dirname(tgt_image), os.path.basename(tgt_image).replace(".tiff", "-reprojected.tiff"))
# reproject_tif(tgt_image, repreojected_tgt_path, rasterio.open(ref_image).profile["crs"])
# tgt_image = repreojected_tgt_path

local_output = os.path.join("../data/outputs/", "L1C_SLC_AN_pair", f"{id}_local.tiff")
global_output = os.path.join("../data/outputs/", "L1C_SLC_AN_pair", f"{id}_global.tiff")

In [ ]:
%%time
coreg_global = COREG(
    im_ref=ref_image,
    im_tgt=tgt_image,
    path_out=global_output,
    fmt_out="GTIFF",
    # v=True,
    nodata=(0.0,0.0),
    r_b4match=2,
    s_b4match=1,
    align_grids=True,
    max_iter = 10,
    max_shift = 10,
    ws = (500, 500)
)
res = coreg_global.correct_shifts()

In [ ]:
%%time
coreg_local = COREG_LOCAL(
    im_ref=ref_image,
    im_tgt=tgt_image,
    grid_res=250,
    # max_points=200,
    path_out=local_output,
    fmt_out="GTIFF",
    # v=True,
    nodata=(0.0,0.0),
    # r_b4match=2,
    # s_b4match=2,
    align_grids=True,
    max_iter = 10,
    max_shift = 10,
    CPUs = 8,
)
res = coreg_local.correct_shifts()